In [182]:
import json
import requests
import urllib.parse
import pandas as pd 
import io
import re

In [232]:
def get_transactions(identifier) :
  '''
    Returns the dataframe with the value of we sent as an indetifier
            Parameters:
                    identifier (string): key
                  

            Returns:
                     ts(dataframe): data with given indentifer
   '''

  # Building blocks for the URL
  entrypoint = 'https://sdw-wsrest.ecb.europa.eu/service/' # Using protocol 'https'
  resource = 'data'           # The resource for data queries is always'data'
  flowRef ='BP6'              # Dataflow describing the data that needs to be returned, exchange rates in this case
  key = identifier
  # key = 'Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N'    # Defining the dimension values, explained below

  # Define the parameters
  parameters = {'detail':"dataonly"}
  # Construct the URL: https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.CHF.EUR.SP00.A
  request_url = entrypoint + resource + '/'+ flowRef + '/' + key

  # Make the HTTP request
  response = requests.get(request_url, params=parameters)

  response = requests.get(request_url, params=parameters, headers={'Accept': 'text/csv'})
  df = pd.read_csv(io.StringIO(response.text))
  
  ts = df.filter(['KEY','TIME_PERIOD', 'OBS_VALUE',], axis=1)
  ts['KEY'] = ts['KEY'].str[4:]
  return ts

  
data_frame =  get_transactions('Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N')




In [249]:
def get_formula_data(formula):
  '''
    Returns the dataframe with the value of we sent as an indetifier
            Parameters:
                    formula (string): string to validate the formaula
            Returns:
                     ts(dataframe): data with given indentifer
  '''
  formula=formula.strip()
  #Validating the expression
  right_side_exp, left_side_data = formula.split("=")[0], formula.split("=")[1] # splitting formula to get right side expression
  left_side_data = re.split('\+|\-', left_side_data)  # splitting left to get single expression

  nuclear_right_exp = re.split('\.',right_side_exp)
  
  df_result = pd.DataFrame()
  for single_exp in left_side_data:
    nuclear_left_exp = re.split('\.',single_exp)
    if nuclear_right_exp[:4] == nuclear_left_exp[:4] and nuclear_right_exp[5:] == nuclear_left_exp[5:]\
     and nuclear_right_exp[4].split()[0]== nuclear_left_exp[4].split()[0][:2]:
     #print(data_frame.loc[data_frame['KEY'] == right_side_exp])
     df_single_result = get_transactions(single_exp)
     df_result = df_result.append(df_single_result)
    else:
      print("Formaula is not valid")

  df2 =  df_result.pivot(index='TIME_PERIOD', columns='KEY')
  df2.columns = [' '.join(col).strip() for col in df2.columns.values]
  return df2


result_df = get_formula_data("Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N=Q.N.I8.W1.S1P.S1.T.A.FA.D.F._Z.EUR._T._X.N+Q.N.I8.W1.S1Q.S1.T.A.FA.D.F._Z.EUR._T._X.N")



In [293]:
def compute_aggregates(formula):
  '''
    Returns the dataframe with the value of we sent as an indetifier
            Parameters:
                    formula (string): string to validate the formaula
            Returns:
                     ts(dataframe): data with given indentifer
  '''
  operator_list =[]
  for i in formula:
    if i == '+' or i == '-':
      operator_list.append(i)
  result_df=get_formula_data(formula)


  counter = 0
  for index, rows in result_df.iterrows():
 
    result= str(rows[0])
    temp_oper = operator_list.copy()
  
 
    for row in rows[1:]: 
      counter = counter + 1
      result=str(result)+ temp_oper.pop()+str(row)
 
    result_df.at[index,formula.split("=")[0]] = eval(result)
  
  print(result_df.filter([index,formula.split("=")[0]]))
 
    
  


 

final_result = compute_aggregates("Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N=Q.N.I8.W1.S1P.S1.T.A.FA.D.F._Z.EUR._T._X.N+Q.N.I8.W1.S1Q.S1.T.A.FA.D.F._Z.EUR._T._X.N")


             Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N
TIME_PERIOD                                           
2008-Q1                                  317663.447909
2008-Q2                                  110453.873559
2008-Q3                                  272454.202262
2008-Q4                                   39367.904658
2009-Q1                                  163595.286115
2009-Q2                                    8819.420810
2009-Q3                                  194988.941473
2009-Q4                                  203746.544992
2010-Q1                                  177381.513129
2010-Q2                                  265023.690143
2010-Q3                                  226062.326712
2010-Q4                                   33834.805148
2011-Q1                                  329427.994705
2011-Q2                                  133419.285211
2011-Q3                                  202857.933471
2011-Q4                                  328710.646832
2012-Q1   